In [9]:
import os
import os.path
import sys
import pandas as pd
import numpy as np
from importlib import reload

In [20]:
LOCAL_PATH = 'data'
RAW_TRACE = 'LOL_AI.pcapng'
TCP_TRACE_FEATURE_FILE = 'LOL_AI_tcp_pkt.csv'
UDP_TRACE_FEATURE_FILE = 'LOL_AI_udp_pkt.csv'
TCP_FLOW_FEATURE_FILE = 'LOL_AI_tcp_flow.csv'
UDP_FLOW_FEATURE_FILE = 'LOL_AI_udp_flow.csv'
# BUCKET_NAME = '' # replace with your bucket name
# KEY = '' # replace with your object key

In [21]:
if not os.path.exists(os.path.join(LOCAL_PATH, RAW_TRACE)):
    if not os.path.exists(LOCAL_PATH):
        os.mkdir(LOCAL_PATH)
    
    import boto3
    import botocore

    s3 = boto3.resource('s3')

    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, os.path.join(LOCAL_PATH, RAW_TRACE))
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

In [22]:
# convert raw trace to readable udp and tcp packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.udp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
%time packet_feature.tcp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))

Conversion done
CPU times: user 1.14 s, sys: 87.3 ms, total: 1.23 s
Wall time: 8.23 s
Conversion done
CPU times: user 23.2 ms, sys: 17 ms, total: 40.2 ms
Wall time: 6.69 s


In [25]:
def to_csv(trace_df, extract_func, file, features, max_packets_per_flow):
    df = extract_func(trace_df, 1.0, upsampled=True, max_packets_per_flow=max_packets_per_flow)
    if df.shape[0] == 0:
        pd.DataFrame(columns=features).to_csv(file, index=False)
    else:
        df[features].to_csv(file, index=False)

In [26]:
from python import flow_feature
reload(flow_feature)

FEATURES = ['avg(pkt_len)', 'stddev(pkt_len)', 'fb_ratio', 'inter_arrival_time', 'pkt_count']

tcp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))
udp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
for trace_df in [tcp_trace_df, udp_trace_df]:
    if tcp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.tcp_generate, os.path.join(LOCAL_PATH, TCP_FLOW_FEATURE_FILE), FEATURES, max_packets_per_flow=1000)
    elif udp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.udp_generate, os.path.join(LOCAL_PATH, UDP_FLOW_FEATURE_FILE), FEATURES, max_packets_per_flow=1000)
    else:
        raise

tcp flows, max pkt per flow->1000: 1it [00:00, 684.90it/s]
udp flows, max pkt per flow->1000: 100%|██████████| 90721/90721 [00:18<00:00, 5001.82it/s]
